# $U(1)$ Gauge Model using L2HMC in graph mode

--------------------------------------------------------------------
### TODO:
* [ ] Look at performance on Cooley (longer training runs).
* [ ] Fit observables to Eq. \ref{eq:therm_time} to determine the thermalization time $\tau$.

\begin{equation} 
f(t) \equiv A \exp^{-t / \tau}+ \,\, B
\label{eq:therm_time}
\end{equation}
* [ ] Look at defining a distance metric as the difference in topological charge between two samples and see what effect adding this as an additional term to the loss function has on the models' ability to tunnel between topological sectors.

--------------------------------------------------------------------

In [1]:
import os
import sys
import time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.special import i0, i1

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.data_utils import (
    calc_avg_vals_errors, block_resampling, jackknife_err
)

%autoreload 2
%matplotlib notebook

#from l2hmc_eager import gauge_dynamics_eager as gde

In [2]:
#tf.enable_eager_execution()
tfe = tf.contrib.eager

#from u1_model_eager import GaugeModelEager, train_one_iter
from gauge_model import GaugeModel

import utils.gauge_model_helpers as helpers

Using TensorFlow backend.


In [3]:
def autocorr(x):
    result = np.correlate(x, x, mode='full')
    result /= result[result.argmax()]
    return result[result.size//2:]

## Define parameters

In [4]:
params = {
    'time_size': 8,
    'space_size': 8,
    'link_type': 'U1',
    'dim': 2,
    'beta': 1.,
    'beta_init': 1.,
    'beta_final': 8.,
    'num_samples': 5,
    'num_steps': 5,
    'eps': 0.1,
    'loss_scale': 0.1,
    'loss_eps': 1e-4,
    'learning_rate_init': 1e-4,
    'learning_rate_decay_steps': 100,
    'learning_rate_decay_rate': 0.98,
    'train_steps': 5000,
    'save_steps': 1000,
    'logging_steps': 50,
    'annealing_steps': 50,
    'annealing_factor': 0.97,
    'clip_value': 10.,
    'rand': False,
    'metric': 'cos2',
    'training_samples_steps': 500,
    'training_samples_length': 100
}

## Create model

In [6]:
config = tf.ConfigProto()
tf.reset_default_graph()

In [ ]:
model = GaugeModel(params=params, 
                   config=config,
                   sess=None,
                   conv_net=True,
                   hmc=False,
                   log_dir=None,
                   restore=False,
                   eps_trainable=True,
                   aux=False,
                   annealing=True,
                   clip_grads=False)

Creating directory for new run: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_142/
Creating directory: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_142/samples_history.
Creating directory: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_142/samples_history/training.
Creating directory: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_142/train_samples.
Creating directory: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_142/annealing_samples.
log_dir: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_142/
info_dir: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_142/run_info/
figs_dir: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_142/figures/
conv_net: True
hmc: False
aux: False
eps_trainable: True
clip_grads: False
annealing: True
beta: 1.0
time_size: 8
space_size: 8
link_type: U1
dim: 2
beta_init: 1.0
beta_final: 8.0
num_samples: 5
num_steps: 5
eps: 0.1
loss_scale: 0.1
loss_eps: 0.0001
learning_rate_init: 0.0001
learning_rate_decay_steps: 100
learning_rate_decay_rate: 0.98
train_steps: 5

In [ ]:
model.train(model.train_steps, kill_sess=False)

In [46]:
run_steps = [100, 200, 500, 1000, 5000]#, 10000]

In [47]:
model.beta_final

8.0

In [48]:
for steps in run_steps:
    _ = model.run(steps)

Running (trained) L2HMC sampler for 100 steps...
step:      0/100    	 time/step (normalized): 4.337e-05
step:     10/100    	 time/step (normalized): 3.36e-05
step:     20/100    	 time/step (normalized): 2.949e-05
step:     30/100    	 time/step (normalized): 3.003e-05
step:     40/100    	 time/step (normalized): 3.089e-05
step:     50/100    	 time/step (normalized): 3.031e-05
step:     60/100    	 time/step (normalized): 3.103e-05
step:     70/100    	 time/step (normalized): 2.82e-05
step:     80/100    	 time/step (normalized): 3.468e-05
step:     90/100    	 time/step (normalized): 3.699e-05
done.
Samples saved to: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_139/samples_history/samples_history_100.pkl.
--------------------------------------------------------------------------------

Running (trained) L2HMC sampler for 200 steps...
step:      0/200    	 time/step (normalized): 3.504e-05
step:     10/200    	 time/step (normalized): 3.983e-05
step:     20/200    	 time/step (n

step:    530/1000   	 time/step (normalized): 3.212e-05
step:    540/1000   	 time/step (normalized): 3.185e-05
step:    550/1000   	 time/step (normalized): 3.124e-05
step:    560/1000   	 time/step (normalized): 3.132e-05
step:    570/1000   	 time/step (normalized): 3.073e-05
step:    580/1000   	 time/step (normalized): 3.194e-05
step:    590/1000   	 time/step (normalized): 3.208e-05
step:    600/1000   	 time/step (normalized): 3.102e-05
step:    610/1000   	 time/step (normalized): 3.114e-05
step:    620/1000   	 time/step (normalized): 3.165e-05
step:    630/1000   	 time/step (normalized): 3.159e-05
step:    640/1000   	 time/step (normalized): 3.191e-05
step:    650/1000   	 time/step (normalized): 3.261e-05
step:    660/1000   	 time/step (normalized): 3.07e-05
step:    670/1000   	 time/step (normalized): 3.36e-05
step:    680/1000   	 time/step (normalized): 3.127e-05
step:    690/1000   	 time/step (normalized): 3.21e-05
step:    700/1000   	 time/step (normalized): 3.221

step:    960/5000   	 time/step (normalized): 3.351e-05
step:    970/5000   	 time/step (normalized): 3.137e-05
step:    980/5000   	 time/step (normalized): 0.0001686
step:    990/5000   	 time/step (normalized): 4.769e-05
step:   1000/5000   	 time/step (normalized): 3.407e-05
step:   1010/5000   	 time/step (normalized): 3.283e-05
step:   1020/5000   	 time/step (normalized): 3.18e-05
step:   1030/5000   	 time/step (normalized): 3.202e-05
step:   1040/5000   	 time/step (normalized): 3.292e-05
step:   1050/5000   	 time/step (normalized): 3.17e-05
step:   1060/5000   	 time/step (normalized): 3.245e-05
step:   1070/5000   	 time/step (normalized): 2.851e-05
step:   1080/5000   	 time/step (normalized): 3.159e-05
step:   1090/5000   	 time/step (normalized): 3.25e-05
step:   1100/5000   	 time/step (normalized): 3.213e-05
step:   1110/5000   	 time/step (normalized): 3.119e-05
step:   1120/5000   	 time/step (normalized): 3.19e-05
step:   1130/5000   	 time/step (normalized): 3.123e

step:   2430/5000   	 time/step (normalized): 3.141e-05
step:   2440/5000   	 time/step (normalized): 3.165e-05
step:   2450/5000   	 time/step (normalized): 3.213e-05
step:   2460/5000   	 time/step (normalized): 3.191e-05
step:   2470/5000   	 time/step (normalized): 3.278e-05
step:   2480/5000   	 time/step (normalized): 3.155e-05
step:   2490/5000   	 time/step (normalized): 3.2e-05
step:   2500/5000   	 time/step (normalized): 3.11e-05
step:   2510/5000   	 time/step (normalized): 3.249e-05
step:   2520/5000   	 time/step (normalized): 2.945e-05
step:   2530/5000   	 time/step (normalized): 3.269e-05
step:   2540/5000   	 time/step (normalized): 3.685e-05
step:   2550/5000   	 time/step (normalized): 3.304e-05
step:   2560/5000   	 time/step (normalized): 3.322e-05
step:   2570/5000   	 time/step (normalized): 3.212e-05
step:   2580/5000   	 time/step (normalized): 3.379e-05
step:   2590/5000   	 time/step (normalized): 3.204e-05
step:   2600/5000   	 time/step (normalized): 3.355

step:   3900/5000   	 time/step (normalized): 3.324e-05
step:   3910/5000   	 time/step (normalized): 3.224e-05
step:   3920/5000   	 time/step (normalized): 3.113e-05
step:   3930/5000   	 time/step (normalized): 3.187e-05
step:   3940/5000   	 time/step (normalized): 3.135e-05
step:   3950/5000   	 time/step (normalized): 3.25e-05
step:   3960/5000   	 time/step (normalized): 3.18e-05
step:   3970/5000   	 time/step (normalized): 3.092e-05
step:   3980/5000   	 time/step (normalized): 3.138e-05
step:   3990/5000   	 time/step (normalized): 3.26e-05
step:   4000/5000   	 time/step (normalized): 3.09e-05
step:   4010/5000   	 time/step (normalized): 3.149e-05
step:   4020/5000   	 time/step (normalized): 3.135e-05
step:   4030/5000   	 time/step (normalized): 3.347e-05
step:   4040/5000   	 time/step (normalized): 3.203e-05
step:   4050/5000   	 time/step (normalized): 3.16e-05
step:   4060/5000   	 time/step (normalized): 3.226e-05
step:   4070/5000   	 time/step (normalized): 3.426e-

In [39]:
import pickle

with open(model.files['parameters_pkl_file'], 'wb') as f:
    pickle.dump(model.params, f)

In [ ]:
model.sess.graph.collections

In [ ]:
model.sess.graph.get_collection

In [ ]:
model.dynamics.position_fn.summary()

In [ ]:
# Iterate over samples history and calculate observables for each sample.
# `lattice.calc_plaq_observables(samples)` calculates observables for each of
# the samples in the mini-batch.
actions_history = []
avg_plaquettes_history = []
top_charges_history = []
for idx, samples in enumerate(samples_history):
    t0 = time.time()
    observables = np.array(model.lattice.calc_plaq_observables(samples))
    actions, plaqs, charges = observables
    
    actions_history.append(actions)
    avg_plaquettes_history.append(plaqs)
    top_charges_history.append(charges)
    
    print(f'step: {idx}  '
          f'time / step: {time.time() - t0:^6.4g}  '
          f'avg action: {np.mean(actions):^6.4g}  '
          f'avg plaquette: {np.mean(plaqs):^6.4g} '
          f'top charge: {np.mean(charges):^6.4g}')

In [ ]:
_ = helpers.plot_run_data(model.data, 
                          model.params, 
                          model.steps_arr, 
                          model.figs_dir, 
                          skip_steps=1)

In [ ]:
#tf.reset_default_graph()
#model = GaugeModel(params=params,
#                   config=None,
#                   sess=None,
#                   conv_net=False,
#                   hmc=False,
#                   log_dir='../../gauge_logs_graph/run_25',
#                   restore=True)

In [ ]:
samples = np.random.randn(*model.samples.shape)
samples_history = []

In [ ]:
for i in range(500):
    t0 = time.time()
    samples = model.sess.run(model.x_out, feed_dict={model.x: samples})
    samples_history.append(samples)
    print(f'step: {i:^6.4g} time/step: {time.time() - t0:^6.4g}')

In [ ]:
samples_history_conv = np.array(samples_history_conv)
print(samples_history_conv.shape)

In [ ]:
import pickle
samples_history_file = os.path.join(model.info_dir, 'samples_history.pkl')
with open(samples_history_file, 'wb') as f:
    pickle.dump(samples_history_conv, f)